In [ ]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table
from jupyter_dash import JupyterDash
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots

In [ ]:
'''
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1(children='Life', style={'textAlign':'center'}),
    dcc.Dropdown(["qx", "lx"], 'qx', id='dropdown-selection'),
    dcc.Graph(id='graph-content', figure=get_age_life_fig("qx"))
])
@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    return get_age_life_fig(value)
app.run_server(debug=True)
'''

In [ ]:
income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")

In [ ]:
income_life_exp.head(3)

In [ ]:
#https://community.plotly.com/t/slider-with-play-button-for-animations-independent-of-plotly/53188/7

In [ ]:
def usa_map():
    df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

    for col in df.columns:
        df[col] = df[col].astype(str)

    df['text'] = df['state'] + '<br>' + \
        'Beef ' + df['beef'] + ' Dairy ' + df['dairy'] + '<br>' + \
        'Fruits ' + df['total fruits'] + ' Veggies ' + df['total veggies'] + '<br>' + \
        'Wheat ' + df['wheat'] + ' Corn ' + df['corn']

    fig = go.Figure(data=go.Choropleth(
        locations=df['code'],
        z=df['total exports'].astype(float),
        locationmode='USA-states',
        colorscale='Reds',
        autocolorscale=False,
        text=df['text'], # hover text
        marker_line_color='white', # line markers between states
        colorbar_title="Millions USD"
    ))

    fig.update_layout(
        title_text='2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            showlakes=True, # lakes
            lakecolor='rgb(255, 255, 255)'),
    )
    return fig

In [ ]:
def life_exp_map():
    df = pd.read_csv(r'./data/Life/LifeExp/StateLifeExp2020CDC.csv')
    fig = go.Figure(data=go.Choropleth(
        locations = df["STATE"],
        z = df["RATE"],
        locationmode = 'USA-states',
        colorscale='Inferno',
        colorbar_title="Life Expectancy"    
    ))
    fig.update_layout(
        title_text ="2020 Life Expectancy - State Wide",
        geo_scope='usa',
    )
    return fig

In [ ]:
app = JupyterDash(__name__)

income_life_exp = pd.read_csv(r"data\Life\health_ineq_online_table_2.csv")



app.layout = html.Div(className="row-fluid", children=[
    html.H1(children='USA', style={'textAlign':'center'}),
    html.Div(className="container-fluid", children=[
        html.Div(className = "row", children=[
            html.Div(dcc.Dropdown(['Life', 'Health', 'Substance', 'Wealth', 'Inequality', 'Enviroment', 
                                   'Peace', 'Safety', 'Terrorism', 'Democracy', 'Equality', 'Knowledge', 
                                   'Quality of Life', 'Happiness']),className = "span8 offset2"),
             
        ]),
        html.Div(className = "row", children=[
            html.Div(dcc.RadioItems(["State", "County "], 'State', inline=True), className="span4 offset5")
        ])
        ]),
   
    html.Div(className = "container-fluid", children=[
        html.Div(children=[
            html.Div(dcc.Graph(id='graph-content', figure=life_exp_map()), className = "span7"),
            html.Div(dash_table.DataTable(id='click-data', page_size=10), className = "span5")])
    ])  
])

@callback(
    Output('click-data', 'data'),
    Input('graph-content', 'clickData'),
    )
def update_click(clickData):
    df = pd.read_csv(r'./data/Life/LifeExp/StateLifeExp2020CDC.csv')
    #df = df[df["code"] == "AZ"]
    try:
        df = df[df["STATE"] == clickData["points"][0]["location"]]
    except:
        print(clickData)
    return df[["RATE","YEAR"]].to_dict('records')


if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
a = {'points': [{'curveNumber': 0, 'pointNumber': 26, 'pointIndex': 26, 'location': 'NE', 'z': 7114.13, 'text': 'Nebraska<br>Beef 762.2 Dairy 30.07<br>Fruits 2.16 Veggies 53.5<br>Wheat 292.3 Corn 1735.9', 'bbox': {'x0': 894.0050665160245, 'x1': 894.0050665160245, 'y0': 286.1820161686125, 'y1': 286.1820161686125}}]}

In [ ]:
a

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
df.to_dict('records')

In [ ]:
income_life_exp


In [ ]:
years = list(income_life_exp.year.unique())
index = years.index(2001)
index = (index + 1) % len(years)
year = years[index]

In [ ]:
year

In [ ]:
https://dash.plotly.com/interactive-graphing